In [1]:
import pandas as pd
import numpy as np

from IPython.display import display, HTML
display(HTML("<style>.container { width:50% !important; }</style>"))

In [2]:
data = pd.read_csv('../data/loan_data.csv') 

train_data = data.sample(frac=0.8)
test_data = data.drop(train_data.index)

In [3]:
discrete_features = ['person_home_ownership','person_gender','person_education','loan_intent','previous_loan_defaults_on_file','loan_status']
continuous_features = ['person_age','person_income','person_emp_exp','loan_amnt','loan_int_rate','loan_percent_income','cb_person_cred_hist_length','credit_score']

In [4]:
train_data[discrete_features].head(5)

person_home_ownership person_gender person_education loan_intent  \
16743              MORTGAGE          male           Master   EDUCATION   
14879                   OWN          male           Master     VENTURE   
13061                   OWN          male         Bachelor   EDUCATION   
21226                  RENT        female        Doctorate     MEDICAL   
26807                   OWN          male           Master    PERSONAL   

      previous_loan_defaults_on_file  loan_status  
16743                            Yes            0  
14879                            Yes            0  
13061                             No            0  
21226                             No            0  
26807                            Yes            0

In [5]:
train_data[continuous_features].head(5)

person_age  person_income  person_emp_exp  loan_amnt  loan_int_rate  \
16743        26.0        91218.0               0    12000.0           6.92   
14879        21.0       142630.0               2    14000.0           7.88   
13061        21.0       106128.0               0    12000.0          11.01   
21226        33.0       137660.0               9     5000.0          11.83   
26807        28.0        33919.0               7     2000.0          11.12   

       loan_percent_income  cb_person_cred_hist_length  credit_score  
16743                 0.13                         4.0           685  
14879                 0.10                         3.0           598  
13061                 0.11                         2.0           614  
21226                 0.04                         9.0           706  
26807                 0.06                         8.0           562

In [6]:
approved_df = train_data[train_data['loan_status'] == 1]
rejected_df = train_data[train_data['loan_status'] == 0]

approved_discrete_features = {feature: approved_df[feature].value_counts(normalize=True) for feature in discrete_features}
rejected_discrete_features = {feature: rejected_df[feature].value_counts(normalize=True) for feature in discrete_features}

approved_continuous_features = {feature:(approved_df[feature].mean(), approved_df[feature].std()) for feature in continuous_features}
rejected_continuous_features = {feature:(rejected_df[feature].mean(), rejected_df[feature].std()) for feature in continuous_features}

In [7]:
normal_distribution = lambda x, mean, std: 1/std/np.sqrt(2*np.pi)*np.exp(-0.5*(x-mean)**2/std**2)

In [8]:
for idx, sample_data in test_data.iterrows():
    likelihood_approved = 0
    likelihood_rejected = 0
    for feature in discrete_features:
        if feature != 'loan_status':
            try:
                likelihood_approved += np.log2(approved_discrete_features[feature][sample_data[feature]])
            except:
                likelihood_approved += -300

            try:
                likelihood_rejected += np.log2(rejected_discrete_features[feature][sample_data[feature]])
            except:
                likelihood_rejected += -300

    for feature in continuous_features:
        (mean, stdev) = approved_continuous_features[feature]
        likelihood_approved += np.log2(normal_distribution(sample_data[feature], mean, stdev))

        (mean, stdev) = rejected_continuous_features[feature]
        likelihood_rejected += np.log2(normal_distribution(sample_data[feature], mean, stdev))

    likelihood_approved += np.log2(len(approved_df) / len(train_data))
    likelihood_rejected += np.log2(len(rejected_df) / len(train_data))

    if likelihood_approved > likelihood_rejected:
        test_data.loc[idx, 'predicted_loan_status'] = 1
    else:
        test_data.loc[idx, 'predicted_loan_status'] = 0

In [9]:
### out of the positive predictions, how many were correct
precision = (test_data.loan_status == test_data.predicted_loan_status).sum()/len(test_data)

### percentage of positives that are correctly predicted
sensitivity = (test_data[(test_data.loan_status == 1) & (test_data.predicted_loan_status == 1)].shape[0] / test_data[test_data.loan_status == 1].shape[0])

### percentage of negatives that are correctly predicted
specificity = (test_data[(test_data.loan_status == 0) & (test_data.predicted_loan_status == 0)].shape[0] / test_data[test_data.loan_status == 0].shape[0])

In [10]:
print(f"Precision: {precision:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

Precision: 0.8716
Sensitivity: 0.7776
Specificity: 0.8989
